https://twitter.com/0x94305/status/1736894774031065093

This notebook models the loss of LPs to arbitrageurs as a function of the fixed gas cost

In [1]:
import numpy as np
from math import sqrt
from numba import njit, float64
from numba.experimental import jitclass

In [35]:
spec = [
    ('sqrtP', float64),
    ('L', float64),
    ('fee', float64)
]

@jitclass(spec)
class AMM:

     def __init__(self, fee=0.0, p = 1000.0, liquidityPerBP = 50_000.00):
         """Constructor, instantiates a new Uniswap-like AMM pool.

         Args:
             fee (float, optional): Pool fee. Defaults to 0.0.
             p (float, optional): Price of token X (eg, ETH) in token Y (eg, USDT). Defaults to 1000.0.
             liquidityPerBP (_type_, optional): The amount of liquidity per basis point, in token Y. Defaults to 50_000.0.
         """

         self.sqrtP = sqrt(p)
         
         # self.L = liquidityPerBP / (self.sqrtP * (sqrt(1.0001) - 1))
         self.L = liquidityPerBP / (self.sqrtP * (sqrt(1.0001) - 1))
         self.fee = fee


     def xReserves(self):
         """Gets reserves of X (base) token in the pool.
        
         The reserves do not include fees and are calculated in Uniswap v2-equivalent units (full range liquidity).


         Returns:
             float: Reserves of X.
         """
         return self.L/self.sqrtP

     def yReserves(self):
         """Gets reserves of Y (quote) token in the pool. 

         The reserves do not include fees and are calculated in Uniswap v2-equivalent units (full range liquidity).

         Returns:
             float: Reserves of Y
         """        
         return self.L*self.sqrtP

     def tradeToPriceWithGasFee(self, targetP, gas=0.0):
         """Attempts to perform an arbitrage swap given the market price and gas fee.

         Args:
             targetP (float): The efficient price towards which the trade will be performed. If it exceeds the current pool price of X, 
                the function will attempt to buy X for the client (the pool will be selling X).
                The swap must be profitable to the client after the swap fee and gas cost. 
             gas (float, optional): total gas cost of the transaction in Y tokens. Defaults to 0.0.

         Returns:
             x, y, fee: If a profitable arbitrage opportunity is found, then it is executed
                against the pool. Return values x and y are positive if the client receives 
                the corresponding amount and negative otherwise. x and y already include the swap fee,
                but not the gas fee. The third return, fee, is for informational purposes and is measured in Y tokens.
                If no profitable swap is found, the state of the pool is unchanged, and the function 
                returns three zeros. 
         """
         currentP = self.sqrtP**2

         if (currentP/(1 - self.fee) > targetP) and (currentP*(1 - self.fee) < targetP):
            # target price is within the current bid-ask spread, no arb opportunity available
            (x, y, fee) = (0,0,0)
            newSqrtP = self.sqrtP

         elif (currentP/(1 - self.fee) < targetP):
            # target price is higher than best ask, try buying X for the client (the pool sells)
            newSqrtP = sqrt(targetP * (1 - self.fee))
            y = -(newSqrtP - self.sqrtP)*self.L
            (x, y, fee) = ((newSqrtP-self.sqrtP)*self.L/(self.sqrtP*newSqrtP),
             y/(1 - self.fee), -y * self.fee * (1 - self.fee))
         else:
            newSqrtP = sqrt(targetP / (1 - self.fee))
            y = -(newSqrtP - self.sqrtP)*self.L
            (x, y, fee) = ((newSqrtP-self.sqrtP)*self.L/(self.sqrtP*newSqrtP),
             y*(1 - self.fee), y*self.fee)

         if (x*targetP + y < gas):
            # The arb opportunity does not justify the gas fee
            (x, y, fee) = (0.0, 0.0, 0.0) 
         else:
            self.sqrtP = newSqrtP

         return (x, y, fee)
     
     def bidAskSpread(self):
         """Returns the bid-ask spread of the AMM.

         Returns:
             bid, ask: the best bid and ask (after-fee) prices offered by the AMM.
         """
         currentP = self.sqrtP**2

         return (currentP*(1 - self.fee), currentP/(1 - self.fee))



In [3]:
@njit
def run_sims2(fee, price, daily_std, blocks_per_day, 
                    days, paths, low_gas_cost, high_gas_cost, liquidityPerBP):
    # This array will store six values for each simulated price path:
    # (0) lvr (as a positive number) - low gas cost, (1) arb's gain (negative) - low gas  cost, (2) total gas burned at low cost
    # (3) lvr (as a positive number) - high gas cost, (4) arb's gain (negative) - high gas  cost, (5) total gas burned at high cost
    results = np.zeros((6, paths))
    
    for jj in range(paths):
    
        # save the initial price
        p0 = price
        
        sigma = daily_std/np.sqrt(blocks_per_day) # vol between blocks
        T = int(days*blocks_per_day)

        # Generate a GBM path for prices
        z = np.cumsum(np.random.normal(0.0, sigma, T))
        # Note that we are adding a risk-neutral drift, so that the price process is a martingale
        prices = np.exp(z-(np.arange(T)*sigma**2)/2)
        prices = prices/prices[0]*p0

        amm = AMM(fee, price, liquidityPerBP)
        amm_high_gas = AMM(fee, price, liquidityPerBP)

        # save the initial reserves   
        initial_yReserves = amm.yReserves()

        lvr0 = arb_gain0 = lvr1 = arb_gain1 = low_gas = high_gas = 0.0
            
        for i in range(1, T):
            x0, y0, f = amm.tradeToPriceWithGasFee(prices[i], low_gas_cost)
            lvr0 += -x0*prices[i] - y0
            if x0 != 0.0: 
                arb_gain0 += x0*prices[i] + y0 - low_gas_cost
                low_gas += low_gas_cost

            x0, y0, f = amm_high_gas.tradeToPriceWithGasFee(prices[i], high_gas_cost)
            lvr1 += -x0*prices[i] - y0
            if x0 != 0.0: 
                arb_gain1 += x0*prices[i] + y0 - high_gas_cost
                high_gas += high_gas_cost

        results[:, jj] = [ lvr0/days, arb_gain0/days, low_gas/days,
                          lvr1/days, arb_gain1/days, high_gas/days]

    return results

In [37]:
fee = 0.0005

initialPrice = 2000.0
daily_std = 0.05 # daily std of the price of X in Y
blocks_per_day = 24.0*60*5 # 5 blocks per minute = 12ss blocks
days = 1.0 # how many days each simulation runs
paths = 10_000 # how many price paths to simulate. days*paths = total number of simulated days
low_gas_cost = 10.0 # low gas fee per swap
high_gas_cost = 10.50 # low gas fee per swap
liquidityPerBPS = 50_000.0 #with a starting price of ETH at $1200 (which is the default value), this L is equivalent to $60,000/bps

res = run_sims2(fee, initialPrice, daily_std, 
                    blocks_per_day, days, paths, low_gas_cost, high_gas_cost, liquidityPerBPS)

l = np.mean(res, 1)

In [38]:
print("Extra gas cost as a share of original gas cost: ", (l[5] - l[2])/l[2])
print("Share of the extra gas cost covered by LPs: ", (l[0] - l[3])/(l[5] - l[2]))
print("Extra loss of LPs as a share of post-fee LVR: ", -(l[0] - l[3])/l[0])
print("Gas cost (w/o events) as a share of post-fee LVR: ", -(l[2])/l[0])

Extra gas cost as a share of original gas cost:  0.040874255611626456
Share of the extra gas cost covered by LPs:  0.5062022195720911
Extra loss of LPs as a share of post-fee LVR:  0.0019357718147344051
Gas cost (w/o events) as a share of post-fee LVR:  0.0935578559359093


In [39]:
fee = 0.0005

initialPrice = 2000.0
daily_std = 0.05 # daily std of the price of X in Y
blocks_per_day = 24.0*60*5 # 5 blocks per minute = 12ss blocks
days = 1.0 # how many days each simulation runs
paths = 10_000 # how many price paths to simulate. days*paths = total number of simulated days
low_gas_cost = 10.0 # low gas fee per swap
high_gas_cost = 10.50 # low gas fee per swap
liquidityPerBPS = 10_000.0 #with a starting price of ETH at $1200 (which is the default value), this L is equivalent to $60,000/bps

res = run_sims2(fee, initialPrice, daily_std, 
                    blocks_per_day, days, paths, low_gas_cost, high_gas_cost, liquidityPerBPS)

l = np.mean(res, 1)

In [40]:
print("Less liquid pair, $10K liquidity per basis point")
print("Extra gas cost as a share of original gas cost: ", (l[5] - l[2])/l[2])
print("Share of the extra gas cost covered by LPs: ", (l[0] - l[3])/(l[5] - l[2]))
print("Extra loss of LPs as a share of post-fee LVR: ", -(l[0] - l[3])/l[0])
print("Gas cost (w/o events) as a share of post-fee LVR: ", -(l[2])/l[0])

Less liquid pair, $10K liquidity per basis point
Extra gas cost as a share of original gas cost:  0.0316581853190822
Share of the extra gas cost covered by LPs:  0.5722192653916589
Extra loss of LPs as a share of post-fee LVR:  0.0049879835939893894
Gas cost (w/o events) as a share of post-fee LVR:  0.27534457480779917


In [41]:
fee = 0.0030

initialPrice = 2000.0
daily_std = 0.05 # daily std of the price of X in Y
blocks_per_day = 24.0*60*5 # 5 blocks per minute = 12ss blocks
days = 1.0 # how many days each simulation runs
paths = 10_000 # how many price paths to simulate. days*paths = total number of simulated days
low_gas_cost = 10.0 # low gas fee per swap
high_gas_cost = 10.50 # low gas fee per swap
liquidityPerBPS = 50_000.0 #with a starting price of ETH at $1200 (which is the default value), this L is equivalent to $60,000/bps

res = run_sims2(fee, initialPrice, daily_std, 
                    blocks_per_day, days, paths, low_gas_cost, high_gas_cost, liquidityPerBPS)

l = np.mean(res, 1)

In [42]:
print("Higher swap fee, 30bps")
print("Extra gas cost as a share of original gas cost: ", (l[5] - l[2])/l[2])
print("Share of the extra gas cost covered by LPs: ", (l[0] - l[3])/(l[5] - l[2]))
print("Extra loss of LPs as a share of post-fee LVR: ", -(l[0] - l[3])/l[0])
print("Gas cost (w/o events) as a share of post-fee LVR: ", -(l[2])/l[0])

Higher swap fee, 30bps
Extra gas cost as a share of original gas cost:  0.04202123802616528
Share of the extra gas cost covered by LPs:  0.7660060234959
Extra loss of LPs as a share of post-fee LVR:  0.0030076846796644125
Gas cost (w/o events) as a share of post-fee LVR:  0.09343966559661301


In [43]:
fee = 0.0005

initialPrice = 2000.0
daily_std = 0.03 # daily std of the price of X in Y
blocks_per_day = 24.0*60*5 # 5 blocks per minute = 12ss blocks
days = 1.0 # how many days each simulation runs
paths = 10_000 # how many price paths to simulate. days*paths = total number of simulated days
low_gas_cost = 10.0 # low gas fee per swap
high_gas_cost = 10.50 # low gas fee per swap
liquidityPerBPS = 50_000.0 #with a starting price of ETH at $1200 (which is the default value), this L is equivalent to $60,000/bps

res = run_sims2(fee, initialPrice, daily_std, 
                    blocks_per_day, days, paths, low_gas_cost, high_gas_cost, liquidityPerBPS)

l = np.mean(res, 1)

In [44]:
print("Lower volatility, 3%")
print("Extra gas cost as a share of original gas cost: ", (l[5] - l[2])/l[2])
print("Share of the extra gas cost covered by LPs: ", (l[0] - l[3])/(l[5] - l[2]))
print("Extra loss of LPs as a share of post-fee LVR: ", -(l[0] - l[3])/l[0])
print("Gas cost (w/o events) as a share of post-fee LVR: ", -(l[2])/l[0])

Lower volatility, 3%
Extra gas cost as a share of original gas cost:  0.036438836357723645
Share of the extra gas cost covered by LPs:  0.6560006879755667
Extra loss of LPs as a share of post-fee LVR:  0.004644010097250102
Gas cost (w/o events) as a share of post-fee LVR:  0.1942783296094141
